# Bollinger Bands
This strategy combines the moving average strategy 4.9 and the volatility sized intervals strategy 4.10. We will provide liquidity inside the Bollinger Bands. These bands are made up of a lower band BOLL = pa −c·v and an upper band BOLU = pa +c·v. The liquidity position will be [pa −v·c,pa +v·c]. This strategy also has an unused leftover token.

Import code dependencies

In [1]:
import math
from datetime import date, timedelta

import pandas as pd

from demeter import (
    TokenInfo,
    Actuator,
    Strategy,
    ChainType,
    PeriodTrigger,
    realized_volatility,
    simple_moving_average,
    MarketInfo,
    RowData,
)
from demeter.result import performance_metrics
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format and const variables.

In [2]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)
c = 2

Custom Add By Volatility strategy with code to add liquidity price at around one day bollinger bands.

In [3]:
class AddByVolatilityStrategy(Strategy):
    """
    We will provide liquidity inside the Bollinger Bands.
    These bands are made up of a lower band BOLL = pa − c · v
    and an upper band BOLU = pa + c · v.
    The liquidity get_position will be [pa − v · c, pa + v · c].

    * pa is simple moving average
    * c is a constant value, =2
    * v is volatility

    we will adjust liquidity every 4 hours, by remove all the liquidity, then even split all the capital into two assets,
    and provide liquidity by the rules above.

    """

    def initialize(self):
        self.add_column(market_key, "sma_1_day", simple_moving_average(self.data[market_key].price, timedelta(days=1)))
        self.add_column(market_key, "volatility", realized_volatility(self.data[market_key].price, timedelta(days=1), timedelta(days=1)))
        self.triggers.append(PeriodTrigger(time_delta=timedelta(hours=4), trigger_immediately=True, do=self.work))
        self.markets.default.even_rebalance(self.data[market_key].price[0])

    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        lp_row_data = row_data.market_status[market_key]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])
        if math.isnan(lp_row_data.volatility):
            return
        limit = c * float(row_data.prices[eth.name]) * lp_row_data.volatility
        lp_market.add_liquidity(lp_row_data.sma_1_day - limit, lp_row_data.sma_1_day + limit)


Main logic to run Actuator, init two token and market with key "market1"

In [4]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("market1")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 5000)
broker.set_balance(eth, 0)

actuator.strategy = AddByVolatilityStrategy()

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
# actuator.run()  # run test

2024-07-22 17:36:28,653 - INFO - start load files from 2023-08-13 to 2023-08-17...
2024-07-22 17:36:28,741 - INFO - load file complete, preparing...
2024-07-22 17:36:29,042 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [5]:
actuator.run()
print({k: round(v, 5) for k, v in performance_metrics(
    actuator.account_status_df["net_value"], benchmark=actuator.account_status_df["price"]["ETH"]
).items()})

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2024-07-22 17:36:29,084 - INFO - init strategy...
C:\Users\sun\AppData\Local\Temp\ipykernel_19304\36816503.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.markets.default.even_rebalance(self.data[market_key].price[0])
2024-07-22 17:36:29,251 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7200/7200 [00:04<00:00, 1715.16it/s]
2024-07-22 17:36:33,465 - INFO - main loop finished
2024-07-22 17:36:33,623 - INFO - Print actuator summary
2024-07-22 17:36:33,785 - INFO - Backtesting finished, execute time 4.7031238079071045s


Final account status                              
Token balance in broker       
USDC      :2426.8888                ETH       :0                        
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
   lower_tick  upper_tick pending0 pending1         liquidity
0      201205      201695        0        0  2409457828585901

Account balance history                           
l1                  net_value    tokens             market1                                                                                          price     
l2                                 USDC       ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count       ETH USDC
2023-08-13 00:00:00 4998.7495 2499.3745 1.3523847         0                0                 0                0                 0           

2024-07-22 17:36:33,879 - INFO - files have saved to ./result\backtest-20240722-173633.account.csv,./result\backtest-20240722-173633.action.json,./result\backtest-20240722-173633.action.pkl


{Return: Decimal('-219.28307'), Rate of Return: Decimal('-0.04387'), Annualized Return: Decimal('-0.96217'), Max Draw Down: Decimal('0.07140'), Sharpe Ratio: Decimal('-14.62548'), Volatility: Decimal('414.47490'), alpha: Decimal('0.00000'), beta: Decimal('0.49466')}


['./result\\backtest-20240722-173633.account.csv',
 './result\\backtest-20240722-173633.action.json',
 './result\\backtest-20240722-173633.action.pkl']